In [1]:
!pip install -e ./gym-env

Obtaining file:///home/naman/courses/cs698/Midsem/gym-env
  Attempting uninstall: gym-env
    Found existing installation: gym-env 1.0.0
    Uninstalling gym-env-1.0.0:
      Successfully uninstalled gym-env-1.0.0
  Running setup.py develop for gym-env


In [1]:
import gym
import numpy as np
import gym_env
import matplotlib.pyplot as plt

In [2]:
rng = np.random.default_rng(0)

In [3]:
# Representation of the MDP for RME
P = {
    0: { # state
        0: [(0.9, 0, -0.04, False), (0.1, 1, -0.04, False)], # action: (p, s', r, f)
        1: [(0.1, 0, -0.04, False), (0.1, 4, -0.04, False), (0.8, 1, -0.04, False)],
        2: [(0.1, 0, -0.04, False), (0.1, 1, -0.04, False), (0.8, 4, -0.04, False)],
        3: [(0.9, 0, -0.04, False), (0.1, 4, -0.04, False)]
    },
    1: {
        0: [(0.1, 0, -0.04, False), (0.1, 2, -0.04, False), (0.8, 1, -0.04, False)],
        1: [(0.2, 1, -0.04, False), (0.8, 2, -0.04, False)],
        2: [(0.1, 0, -0.04, False), (0.1, 2, -0.04, False), (0.8, 1, -0.04, False)],
        3: [(0.2, 1, -0.04, False), (0.8, 0, -0.04, False)],
    },
    2: {
        0: [(0.1, 1, -0.04, False), (0.1, 3, 1, True), (0.8, 2, -0.04, False)],
        1: [(0.1, 2, -0.04, False), (0.8, 3, 1, True), (0.1, 6, -0.04, False)],
        2: [(0.1, 1, -0.04, False), (0.1, 3, 1, True), (0.8, 6, -0.04, False)],
        3: [(0.1, 2, -0.04, False), (0.8, 1, -0.04, False), (0.1, 6, -0.04, False)],
    },
    4: {
        0: [(0.8, 0, -0.04, False), (0.2, 4, -0.04, False)],
        1: [(0.1, 0, -0.04, False), (0.1, 8, -0.04, False), (0.8, 4, -0.04, False)],
        2: [(0.8, 8, -0.04, False), (0.2, 4, -0.04, False)],
        3: [(0.1, 0, -0.04, False), (0.1, 8, -0.04, False), (0.8, 4, -0.04, False)],
    },
    6: {
        0: [(0.1, 6, -0.04, False), (0.1, 7, -1, True), (0.8, 2, -0.04, False)],
        1: [(0.1, 2, -0.04, False), (0.8, 7, -1, True), (0.1, 10, -0.04, False)],
        2: [(0.1, 6, -0.04, False), (0.1, 7, -1, True), (0.8, 10, -0.04, False)],
        3: [(0.1, 2, -0.04, False), (0.8, 6, -0.04, False), (0.1, 10, -0.04, False)],
    },
    8: {
        0: [(0.8, 4, -0.04, False), (0.1, 8, -0.04, False), (0.1, 9, -0.04, False)],
        1: [(0.1, 4, -0.04, False), (0.1, 8, -0.04, False), (0.8, 9, -0.04, False)],
        2: [(0.9, 8, -0.04, False), (0.1, 9, -0.04, False)],
        3: [(0.9, 8, -0.04, False), (0.1, 4, -0.04, False)]
    },
    9: {
        0: [(0.1, 8, -0.04, False), (0.1, 10, -0.04, False), (0.8, 9, -0.04, False)],
        1: [(0.2, 9, -0.04, False), (0.8, 10, -0.04, False)],
        2: [(0.1, 8, -0.04, False), (0.1, 10, -0.04, False), (0.8, 9, -0.04, False)],
        3: [(0.2, 9, -0.04, False), (0.8, 8, -0.04, False)],
    },
    10: {
        0: [(0.1, 9, -0.04, False), (0.1, 11, -0.04, False), (0.8, 6, -0.04, False)],
        1: [(0.1, 10, -0.04, False), (0.1, 6, -0.04, False), (0.8, 11, -0.04, False)],
        2: [(0.1, 9, -0.04, False), (0.1, 11, -0.04, False), (0.8, 10, -0.04, False)],
        3: [(0.1, 10, -0.04, False), (0.1, 6, -0.04, False), (0.8, 9, -0.04, False)],
    },
    11: {
        0: [(0.1, 11, -0.04, False), (0.1, 10, -0.04, False), (0.8, 7, -1, True)],
        1: [(0.1, 7, -1, True), (0.9, 11, -0.04, False)],
        2: [(0.1, 10, -0.04, False), (0.9, 11, -0.04, False)],
        3: [(0.1, 7, -1, True), (0.1, 11, -0.04, False), (0.8, 10, -0.04, False)],
    }
}

In [4]:
def PolicyEvaluation(policy, P, gamma, theta):
    v_old = np.zeros(12)
    non_terminal = [0,1,2,4,6,8,9,10,11]
    iters = 0
    while True:
        iters += 1
        v_new = np.zeros(12)
        for s in non_terminal:
            for a in range(4):
                temp = 0
                for p, snew, r, f in P[s][a]:
                    temp += p * (r + gamma * v_old[snew])
                    # if f == True then v_old[snew] would be 0
                v_new[s] += policy[s][a] * temp
        if np.max(np.abs(v_new - v_old)) < theta:
            break
        v_old = v_new
    print(f'PolicyEvaluation completed in {iters} iterations')
    return v_new

In [5]:
policy = np.zeros((12, 4))
non_terminal = [0,1,2,4,6,8,9,10,11]
policy_actions = [3,3,3,3,1,3,3,1,0]
for i in range(len(non_terminal)):
    policy[non_terminal[i]][policy_actions[i]] = 1

In [6]:
def PolicyImprovement(v, P, gamma):
    policy = np.zeros((12, 4))
    q = np.zeros((12, 4))
    for s in non_terminal:
        for a in range(4):
            for p, snew, r, f in P[s][a]:
                q[s][a] += p * (r + gamma * v[snew])
    
    for s in non_terminal:
        opt_action = np.argmax(q[s])
        policy[s][opt_action] = 1
    
    return policy

In [7]:
def PolicyIteration(P, gamma, theta, policy=None):
    iters = 0
    while True:
        iters += 1
        policy_old = policy
        v = PolicyEvaluation(policy, P, gamma, theta)
        policy = PolicyImprovement(v, P, gamma)
        if np.all(policy_old == policy):
            break
    
    print(f'Converged to optimal policy in {iters} iterations')
    return v, policy

In [13]:
opt_v, opt_policy = PolicyIteration(P, 0.99, 1e-10, policy)
print(opt_v, opt_policy, sep='\n')

PolicyEvaluation completed in 1972 iterations
PolicyEvaluation completed in 175 iterations
PolicyEvaluation completed in 38 iterations
PolicyEvaluation completed in 38 iterations
Converged to optimal policy in 4 iterations
[0.82442985 0.89286374 0.95464233 0.         0.76427487 0.
 0.68820946 0.         0.69763948 0.63906542 0.60613373 0.38186228]
[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]]


In [9]:
def ValueIteration(P, gamma, theta):
    v = np.zeros(12)
    non_terminal = [0,1,2,4,6,8,9,10,11]
    iters = 0
    while True:
        iters += 1
        q = np.zeros((12, 4))
        for s in non_terminal:
            for a in range(4):
                for p, snew, r, f in P[s][a]:
                    q[s][a] += p * (r + gamma * v[snew])
        
        if np.max(np.abs(v - np.max(q, axis=1))) < theta:
            break
        v = np.max(q, axis=1)
    
    policy = np.zeros((12, 4))
    for s in non_terminal:
        a = np.argmax(q[s])
        policy[s][a] = 1
    print(f'Converged to optimal policy in {iters} iterations')

    return v, policy

In [11]:
opt_v, opt_policy = ValueIteration(P, 0.99, 1e-10)
print(opt_v, opt_policy, sep='\n')

Converged to optimal policy in 38 iterations
[0.82442985 0.89286374 0.95464233 0.         0.76427487 0.
 0.68820946 0.         0.69763948 0.63906542 0.60613373 0.38186228]
[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]]
